In [1]:
import pandas as pd
import os

In [2]:
import time
from tqdm import tqdm

In [3]:
! pip3 install opencv-python 

In [4]:
import tensorflow as tf
import cv2
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


In [17]:
movie = pd.read_csv("134MHCP/MovieGenre.csv", encoding='latin-1')

In [5]:
movie.head()

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...


In [21]:
# for i in range(0, len(movie.Poster)):
#     a = movie.Poster[i]
#     b = movie.Title[i]+".jpg"
#     ! wget -O "$b" "$a"

In [ ]:
x = []
for name in tqdm((os.listdir("134MHCP/Images/"))):
    sb = "134MHCP/Images/"+name
    img = cv2.imread(sb)
    x.append(img)
x

In [15]:
a = []
for name in (os.listdir("134MHCP/Images/")):
    sb = os.path.splitext(name)[0]
    a.append(sb)
a
titles = pd.DataFrame(a, columns = ['Title'])

In [18]:
merge = pd.merge(titles,movie, on = ['Title'], left_index = True, right_index = True)
merge

,Title,imdbId,Imdb Link,IMDB Score,Genre,Poster
0,The Letter Writer (2011),114709,http://www.imdb.com/title/tt114709,8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,Dawn Rider (2012),113497,http://www.imdb.com/title/tt113497,6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,Playmobil: The Secret of Pirate Island (2009),113228,http://www.imdb.com/title/tt113228,6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,Yeralti (2012),114885,http://www.imdb.com/title/tt114885,5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,Liam (2000),113041,http://www.imdb.com/title/tt113041,5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...
5,Riten (1969),113277,http://www.imdb.com/title/tt113277,8.2,Action|Crime|Drama,https://images-na.ssl-images-amazon.com/images...
6,Anguish (2015),114319,http://www.imdb.com/title/tt114319,6.3,Comedy|Drama,https://images-na.ssl-images-amazon.com/images...
7,The Betrayed (2008),112302,http://www.imdb.com/title/tt112302,5.6,Adventure|Comedy|Drama,https://images-na.ssl-images-amazon.com/images...
8,Birds of America (2008),114576,http://www.imdb.com/title/tt114576,5.7,Action|Crime|Thriller,https://images-na.ssl-images-amazon.com/images...
9,Over Your Cities Grass Will Grow (2010),113189,http://www.imdb.com/title/tt113189,7.2,Action|Adventure|Thriller,https://images-na.ssl-images-amazon.com/images...


In [19]:
one_hot = pd.get_dummies(movie['Genre'])
df = merge.drop('Genre', axis=1)
df = merge['Genre'].str.join(sep='').str.get_dummies(sep='|')
df

,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,0,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
5,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [20]:
y = df.values

In [ ]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
model = Sequential()
model.add(Conv2D(32, 3, 3, activation='relu', input_shape=(101,100,3)))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(32, 3, 3, activation='relu', input_shape=(101,100,3)))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, 3, 3, activation='relu', input_shape=(101,100,3)))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=32, nb_epoch=100, verbose=1)

In [ ]:
prediction = model.predict(x_test)